In [2]:
import pandas as pd
import numpy as np
from functions import load_data, get_train_targets, get_test_data, prepare_submission, remove_ouliers

data_a, data_b, data_c = load_data()

data_a = remove_ouliers(data_a)
data_b = remove_ouliers(data_b)
data_c = remove_ouliers(data_c)

X_train_a, targets_a = get_train_targets(data_a)
X_train_b, targets_b = get_train_targets(data_b)
X_train_c, targets_c = get_train_targets(data_c)

X_test_a, X_test_b, X_test_c = get_test_data()

In [13]:
locA_features = ['clear_sky_energy_1h:J', 'cloud_base_agl:m', 'dew_point_2m:K', 
                 'diffuse_rad_1h:J', 'direct_rad_1h:J', 'sun_azimuth:d', 'sun_elevation:d', 
                 'wind_speed_u_10m:ms','sfc_pressure:hPa','snow_water:kgm2','precip_5min:mm', 
                 'fresh_snow_3h:cm']

drop_colsA = list(set(list(X_train_a.columns)) - set(locA_features))

log_transformA = ['cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad_1h:J', 'direct_rad_1h:J', 'sun_elevation:d', 
                 'snow_water:kgm2','precip_5min:mm', 'fresh_snow_3h:cm','dew_point_2m:K']

locB_features = ['clear_sky_rad:W', 'cloud_base_agl:m',
                 'diffuse_rad:W', 'direct_rad:W', 'sun_azimuth:d', 'sun_elevation:d', 
                 'wind_speed_u_10m:ms','sfc_pressure:hPa','snow_water:kgm2','precip_5min:mm', 
                 'fresh_snow_3h:cm','dew_point_2m:K']

drop_colsB = list(set(list(X_train_b.columns)) - set(locB_features))


locC_features = ['clear_sky_rad:W', 'cloud_base_agl:m',
                 'diffuse_rad:W', 'direct_rad:W', 'sun_azimuth:d', 'sun_elevation:d', 
                 'wind_speed_u_10m:ms','sfc_pressure:hPa','snow_water:kgm2','precip_5min:mm', 
                 'fresh_snow_3h:cm','dew_point_2m:K']

drop_colsC = list(set(list(X_train_c.columns)) - set(locC_features))

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

class ColumnDropper(BaseEstimator, TransformerMixin):
    """Drops columns from the data."""

    def __init__(self, drop_cols = []):
        self.drop_cols = drop_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        return X_copy.drop(columns=self.drop_cols)
    
class LogTransformer(BaseEstimator, TransformerMixin):
    """Does a log transform on specified columns."""

    def __init__(self, cols = []):
        self.drop_cols = drop_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        return X_copy.drop(columns=self.drop_cols)

# data_process_pipeline = Pipeline([
#     ('drop_cols', ColumnDropper(drop_cols=drop_cols)),
#     ('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
# ])


locA_pipeline = Pipeline([
    #('data_process', data_process_pipeline),
    ('drop_cols', ColumnDropper(drop_cols=drop_colsA)),
    ('log_transform', np.log1p, log_transformA)
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
    ('svr', SVR(kernel='linear'))
])

locB_pipeline = Pipeline([
    #('data_process', data_process_pipeline),
    ('drop_cols', ColumnDropper(drop_cols=drop_colsB)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
    ('random_forest', RandomForestRegressor(bootstrap=False, max_features=0.4, min_samples_leaf=5, min_samples_split=2, n_estimators=100, random_state=1))
])

locC_pipeline = Pipeline([
    #('data_process', data_process_pipeline),
    ('drop_cols', ColumnDropper(drop_cols=drop_colsC)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
    ('random_forest', XGBRegressor(learning_rate=0.1, max_depth=9, min_child_weight=3, n_estimators=100, n_jobs=1, objective="reg:squarederror", subsample=0.8500000000000001, verbosity=0))
])

<>:31: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:31: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
/var/folders/n0/h43nw_h10zg8v_gzjtqbktp80000gn/T/ipykernel_75253/1692105942.py:31: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('log_transform', np.log1p, log_transformA)
/var/folders/n0/h43nw_h10zg8v_gzjtqbktp80000gn/T/ipykernel_75253/1692105942.py:31: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('log_transform', np.log1p, log_transformA)
/var/folders/n0/h43nw_h10zg8v_gzjtqbktp80000gn/T/ipykernel_75253/1692105942.py:31: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('log_transform', np.log1p, log_transformA)
/var/folders/n0/h43nw_h10zg8v_gzjtqbktp80000gn/T/ipykernel_75253/1692105942.py:31: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('log_transform', np.log1p, log_transformA)
/var/folders/n0/h43n

TypeError: 'tuple' object is not callable

In [15]:
locA_pipeline.fit(X_train_a, targets_a)
pred_a = locA_pipeline.predict(X_test_a.drop(columns=["id", "prediction", "location"]))

TypeError: All estimators should implement fit and transform, or can be 'drop' or 'passthrough' specifiers. '<ufunc 'log1p'>' (type <class 'numpy.ufunc'>) doesn't.